In [8]:
print('hi')

hi


In [10]:
from web3 import Web3
from eth_abi.abi import decode
import pandas as pd
from datetime import datetime

In [27]:
# Connect to Ethereum node (using Infura here, replace with your actual Infura project ID)
infura_url = "https://mainnet.infura.io/v3/b5b6a9bfcc4545d88ed6ced14d5c9824"
web3 = Web3(Web3.HTTPProvider(infura_url))

# Check connection
if not web3.is_connected():
    raise Exception("Failed to connect to Ethereum node")

# Replace with your pool contract address (must start with '0x')
pool_address = "0x76e222b07c53d28b89b0bac18602810fc22b49a8"
pool_address = web3.to_checksum_address(pool_address)

# The correct Swap event signature for Uniswap V2 is:
# Swap(address indexed sender, uint amount0In, uint amount1In, uint amount0Out, uint amount1Out, address indexed to)
# So the signature text (without parameter names, with types only) is:
event_signature_text = "Swap(address,address,int256,int256,uint160,uint128,int24)"

# Compute event signature hash (topic0)
event_signature_hash = web3.keccak(text=event_signature_text).hex()

# Make sure address and topic start with '0x'
if not pool_address.startswith("0x"):
    pool_address = "0x" + pool_address

if not event_signature_hash.startswith("0x"):
    event_signature_hash = "0x" + event_signature_hash

# Set the block range you want to fetch logs from
start_block = 22500000
end_block = 22578760  # Keep this small for testing to avoid timeout/errors

# Prepare the filter parameters
filter_params = {
    "fromBlock": start_block,
    "toBlock": end_block,
    "address": pool_address,
    "topics": [event_signature_hash],
}

# Fetch logs
try:
    logs = web3.eth.get_logs(filter_params)
    print(f"Fetched {len(logs)} Swap logs")
    
    # Print the first log (raw data)
    if logs:
        print(logs[0])
except Exception as e:
    print(f"Error fetching logs: {e}")


Fetched 0 Swap logs


In [42]:
from web3 import Web3

# Connect to Ethereum node (Infura)
infura_url = "https://mainnet.infura.io/v3/b5b6a9bfcc4545d88ed6ced14d5c9824"
web3 = Web3(Web3.HTTPProvider(infura_url))

# Check connection
if not web3.is_connected():
    raise Exception("Failed to connect to Ethereum node")

# Pool address (make sure it's checksummed)
pool_address = "0x76e222b07c53d28b89b0bac18602810fc22b49a8"
pool_address = web3.to_checksum_address(pool_address)

# Swap event signature (Uniswap V3)
event_signature_text = "Swap(address,address,int256,int256,uint160,uint128,int24)"
event_signature_hash_bytes = web3.keccak(text=event_signature_text)
event_signature_hash = web3.to_hex(event_signature_hash_bytes)  # this adds 0x prefix

# Block range (adjust as needed)
start_block = 22570000
end_block = 22578760

# Prepare filter parameters with correct checksum and 0x-prefixed topic
filter_params = {
    "fromBlock": start_block,
    "toBlock": end_block,
    "address": pool_address,
    # "topics": [event_signature_hash],  # must be a list of hex strings
}

# logs = web3.eth.get_logs(filter_params)
# print(f"Fetched {len(logs)} total logs from contract")

# # Filter logs locally for Swap events by comparing first topic to event_signature_hash
# swap_logs = [log for log in logs if log['topics'][0].hex() == event_signature_hash]

# print(f"Filtered {len(swap_logs)} Swap logs")

# # Optionally print first 3 swap logs
# for i, log in enumerate(swap_logs[:3]):
#     print(f"\nSwap Log #{i+1}:")
#     print(log)

# try:
#     logs = web3.eth.get_logs(filter_params)
#     print(f"Fetched {len(logs)} Swap logs")

#     for i, log in enumerate(logs[:5]):
#         print(f"\nLog #{i+1}:")
#         print(log)

# except Exception as e:
#     print(f"Error fetching logs: {e}")

# Fetch logs
logs = web3.eth.get_logs(filter_params)
print(f"Fetched {len(logs)} total logs from contract")

# Print unique first topics found (event signatures)
topics_set = set()
for log in logs:
    # topics are list of bytes, convert first topic to hex string
    topics_set.add(log['topics'][0].hex())

print("Unique first topics in fetched logs:")
for topic in topics_set:
    print(topic)

# Now filter logs to only those matching Swap event signature
swap_logs = [log for log in logs if log['topics'][0].hex() == event_signature_hash]
print(f"Filtered {len(swap_logs)} Swap logs")

# To decode Swap event logs, we need the ABI snippet for Swap event
swap_event_abi = {
    "anonymous": False,
    "inputs": [
        {"indexed": True, "internalType": "address", "name": "sender", "type": "address"},
        {"indexed": True, "internalType": "address", "name": "recipient", "type": "address"},
        {"indexed": False, "internalType": "int256", "name": "amount0", "type": "int256"},
        {"indexed": False, "internalType": "int256", "name": "amount1", "type": "int256"},
        {"indexed": False, "internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
        {"indexed": False, "internalType": "uint128", "name": "liquidity", "type": "uint128"},
        {"indexed": False, "internalType": "int24", "name": "tick", "type": "int24"}
    ],
    "name": "Swap",
    "type": "event"
}

# Create a contract object with only the Swap event ABI
contract = web3.eth.contract(address=pool_address, abi=[swap_event_abi])

# Decode Swap logs and print first few decoded events
print("\nDecoded Swap events:")
for i, log in enumerate(swap_logs[:5]):  # show first 5
    decoded = contract.events.Swap().process_log(log)
    print(f"Swap event #{i+1}:")
    print(f"  sender:    {decoded.args.sender}")
    print(f"  recipient: {decoded.args.recipient}")
    print(f"  amount0:   {decoded.args.amount0}")
    print(f"  amount1:   {decoded.args.amount1}")
    print(f"  sqrtPriceX96: {decoded.args.sqrtPriceX96}")
    print(f"  liquidity: {decoded.args.liquidity}")
    print(f"  tick:      {decoded.args.tick}")
    print()


Fetched 345 total logs from contract
Unique first topics in fetched logs:
ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef
8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925
Filtered 0 Swap logs

Decoded Swap events:
